<a href="https://colab.research.google.com/github/ItaiSeri/My_Investment_Analysis/blob/master/invest_test_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
z = '/content/drive/My Drive/Collab Notebooks/MyFunctions'
!pip install ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from glob import glob
#from ipynb.fs.full.z import xirr #Import 'xirr' function from another notebook
from google.colab import drive
drive.mount('/content/drive')


#path = Path("F:/Dowloads/investment/")
path = Path("/content/drive/My Drive/investment/")
#%run '/content/drive/My Drive/Collab Notebooks/MyFunctions.ipynb'


filenames = sorted(path.glob('data*.xlsx')) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Append Excel filenames into one DataFrame


In [3]:
df = pd.DataFrame()
for f in filenames:
    data = pd.read_excel(f)
    df = df.append(data)
df = df.reset_index(drop=True)
df.head(2)

,תאריך,נייר/תנועה,מס' נייר/תנועה,פעולה,כמות,מחיר,זכות נטו,חובה נטו,עמלה,יתרה,אסמכתא
0,30 בנוב׳ 2014,"100 ת""א MTF",5109889,הפקדה,7028.0,144.65,0.0,0.0,0.0,0.0,995021
1,30 בנוב׳ 2014,S&P 500 MTF,5117478,הפקדה,14939.0,149.93,0.0,0.0,0.0,0.0,995022


Translate Columns from Hebrew to English

In [4]:
old_cols = list (df.columns)
new_cols = ['date','stock_desc','stock_num','action_type','quantity','price','action_credit','action_cost','commission','balance','confirmation_num']
new_cols_dict = dict(zip(old_cols, new_cols))
df.rename(columns = new_cols_dict,inplace=True)
df.head(2)

,date,stock_desc,stock_num,action_type,quantity,price,action_credit,action_cost,commission,balance,confirmation_num
0,30 בנוב׳ 2014,"100 ת""א MTF",5109889,הפקדה,7028.0,144.65,0.0,0.0,0.0,0.0,995021
1,30 בנוב׳ 2014,S&P 500 MTF,5117478,הפקדה,14939.0,149.93,0.0,0.0,0.0,0.0,995022


Cast 'date' column to date datatype

In [5]:
months_df = pd.read_excel(path / 'month_translation.xlsx', header = None)
month_dict = dict(zip(months_df.iloc[:,0], months_df.iloc[:,1]))
df['date'] = df['date'].replace(month_dict, regex=True).replace('ב','' ,regex=True).str.replace(" ","")
df['date'] =  pd.to_datetime(df['date'], format='%d%b%Y')
df.head(2)

,date,stock_desc,stock_num,action_type,quantity,price,action_credit,action_cost,commission,balance,confirmation_num
0,2014-11-30,"100 ת""א MTF",5109889,הפקדה,7028.0,144.65,0.0,0.0,0.0,0.0,995021
1,2014-11-30,S&P 500 MTF,5117478,הפקדה,14939.0,149.93,0.0,0.0,0.0,0.0,995022


A few more things

In [6]:
df["stock_desc"] = df.stock_desc.str.split().str.join(" ") #Clean extra white spaces in 'stock_desc' column
df = df[df['action_type'] != 'העברה'] #delete irrelevant rows
df.head(2)

,date,stock_desc,stock_num,action_type,quantity,price,action_credit,action_cost,commission,balance,confirmation_num
0,2014-11-30,"100 ת""א MTF",5109889,הפקדה,7028.0,144.65,0.0,0.0,0.0,0.0,995021
1,2014-11-30,S&P 500 MTF,5117478,הפקדה,14939.0,149.93,0.0,0.0,0.0,0.0,995022


In [0]:
#print (df.loc[0,['action_cost','commission','action_credit']])
#df.loc[0,['action_cost','commission','action_credit']] == 0.all(1)
#df.loc[0,['action_cost','commission','action_credit']].isnull().eq(True, axis=0).all(1)


Get data regarding present state of the portfolio

In [0]:
present_state = pd.read_excel(path / '25062019_snapshot.xlsx') #import table
ps = present_state.loc[:,['שם נייר','מספר נייר','שווי נוכחי']] #leave only relevant columns
ps.columns = ['stock_desc','stock_num','value'] #translate columns names
ps['date'] = pd.Timestamp('25/06/2019') #add date column
ps.head(2)

,stock_desc,stock_num,value,date
0,RU DEPOS.קסם,1145770,17430.7,2019-06-25
1,פסג.תא נדלן,1148691,20510.5,2019-06-25


Concat portfolio present state to main df

In [0]:
df = pd.concat([df,ps.rename(columns={'value':'action_credit'})],axis = 0, sort=False)
df.tail(20)

,date,stock_desc,stock_num,action_type,quantity,price,action_credit,action_cost,commission,balance,confirmation_num
355,2019-03-31,31/03/19 דמי שימוש/ניהול חשבון,900,ד.טפול,0.0,0.0,0.00,24.0,0.0,9756.49,0.0
356,2019-03-31,מגן מס,9992983,הפקדה,6.0,100.0,0.00,0.0,0.0,9756.49,0.0
357,2019-04-30,30/04/19 דמי שימוש/ניהול חשבון,900,ד.טפול,0.0,0.0,0.00,24.0,0.0,9732.49,0.0
358,2019-04-30,מגן מס,9992983,הפקדה,6.0,100.0,0.00,0.0,0.0,9732.49,0.0
359,2019-05-31,31/05/19 דמי שימוש/ניהול חשבון,900,ד.טפול,0.0,0.0,0.00,24.0,0.0,9708.49,0.0
360,2019-05-31,מגן מס,9992983,הפקדה,6.0,100.0,0.00,0.0,0.0,9708.49,0.0
0,2019-06-25,RU DEPOS.קסם,1145770,NaN,NaN,NaN,17430.70,NaN,NaN,NaN,NaN
1,2019-06-25,פסג.תא נדלן,1148691,NaN,NaN,NaN,20510.50,NaN,NaN,NaN,NaN
2,2019-06-25,DAX 30.הרל,1149053,NaN,NaN,NaN,46174.80,NaN,NaN,NaN,NaN
3,2019-06-25,SP500.MTF,1150333,NaN,NaN,NaN,108001.06,NaN,NaN,NaN,NaN


Create df_xirr dataframe in a format that xirr() function acceptes

In [0]:
yahav_df = pd.DataFrame({'date':pd.Timestamp('20/10/2014'), 'value': -120000}, index = [0])
df_cost = df.loc[:,['date','action_cost','commission']]
df_cost['action_cost'] = (df_cost['action_cost'] + df_cost['commission']) *-1
df_cost = df_cost.loc[:,['date','action_cost']]
df_credit = df.loc[:,['date','action_credit']]
df_xirr = pd.concat([yahav_df,df_cost.rename(columns={'action_cost':'value'}),df_credit.rename(columns={'action_credit':'value'})], axis = 0)
df_xirr = df_xirr[df_xirr.value != 0]
df_xirr.head()

,date,value
0,2014-10-20,-120000.00
8,2015-01-13,-751.76
10,2015-01-31,-21.65
12,2015-02-01,-7005.03
13,2015-02-02,-1003.55


In [0]:
#df_xirr.to_excel(path /'df_xirr.xlsx')

In [0]:
tuples_xirr = [tuple(row) for row in df_xirr.values]
xirr(tuples_xirr)*100

In [0]:
tuples_xirr

In [0]:
import sys
print(sys.path)

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']
